In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import csv
import requests
import time
import os
import json

In [9]:
# configure
file_path = '../data/raw/save_answers/questions.txt'
output_path = '../data/load/save_answers'
delay_seconds = 10

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [10]:
list_of_questions = []

with open(file_path, 'rt') as txt_file:
  for text in txt_file:
    text = text.strip()
    if text != '':
        list_of_questions.append(text)

print(list_of_questions)

response = requests.get('https://scripturecentralqa.org/api/search', params={'q': list_of_questions[0]})
res = response.content.decode('utf-8')
resultData = json.loads(res)
resultData["results"] 

['Who is Oliver Cowdery?', 'Who wrote the Book of Mormon?', 'How do the Bible and Book of Mormon work together?', 'Where did the Book of Mormon happen?', 'Where is the Hill Cumorah?', 'What archaeological proof is there for the Book of Mormon?', 'Where did we get the Book of Abraham?', 'How long did it take to translate the Book of Mormon?', 'How was the Book of Mormon translated?', 'What was the Book of Commandments?', 'Why aren’t the Lectures on Faith in the Doctrine and Covenants any more?', 'Where do we learn about the temple in the scriptures?', 'Why do we need to serve missions?', 'Did the Three Witnesses leave the Church?', 'What is the Joseph Smith Translation?', 'What led to the Church discontinuing polygamy?', 'What led to the revelation on the priesthood in 1978?', 'How many women are in the Book of Mormon?', 'Did Emma Smith leave the Church?', 'What is the covenant path?', 'How can we draw unto Christ?', 'Why is the Book of Mormon the most correct book?', 'Did Paul write th

[{'id': '5cccdc579921220156f91ef28369585d6d8976ddf92b41012e7163830146a048',
  'index': 1,
  'title': 'Oliver Cowdery',
  'url': 'https://doctrineandcovenantscentral.org/people-of-the-dc/oliver-cowdery/',
  'text': 'Oliver’s sacrifice to share the message of the Restoration from New York to the western frontier of the United States bespeaks his commitment to the Lord. Of Oliver’s ability to stir the hearts of men to greater good Wilford Woodruff wrote, “I have seen Oliver Cowdery when it seemed as though the earth trembled under his feet. I never heard a man bear a stronger testimony than he did when under the influence of the Spirit.”[2](#f2) No wonder on December 5, 1834 Oliver was ordained an assistant to the Prophet Joseph. On that occasion Joseph said, “I lay my hands upon thee and ordain thee an assistant-president to the High and Holy Priesthood, in the Church of the Latter-day Saints.”[3](#f3) On April 3, 1836 Oliver was with Joseph as priesthood keys were received from ancient 

In [11]:
file_data = {}

for question in list_of_questions:
    list_data = []
    time.sleep(delay_seconds)
    response = requests.get('https://scripturecentralqa.org/api/search', params={'q': question})
    
    res = response.content.decode('utf-8')
    result_data = json.loads(res)
    for item in result_data['results'][:5]:
        result_temp = json.dumps(item)
        list_data.append(result_temp)
        if len(list_data) > 4:
            break
    print(list_data)
    result_set = '\n--------\n'.join(list_data)

    file_data[question] = [result_set,result_data['answer']]
print(file_data)

['{"id": "5cccdc579921220156f91ef28369585d6d8976ddf92b41012e7163830146a048", "index": 1, "title": "Oliver Cowdery", "url": "https://doctrineandcovenantscentral.org/people-of-the-dc/oliver-cowdery/", "text": "Oliver\\u2019s sacrifice to share the message of the Restoration from New York to the western frontier of the United States bespeaks his commitment to the Lord. Of Oliver\\u2019s ability to stir the hearts of men to greater good Wilford Woodruff wrote, \\u201cI have seen Oliver Cowdery when it seemed as though the earth trembled under his feet. I never heard a man bear a stronger testimony than he did when under the influence of the Spirit.\\u201d[2](#f2) No wonder on December 5, 1834 Oliver was ordained an assistant to the Prophet Joseph. On that occasion Joseph said, \\u201cI lay my hands upon thee and ordain thee an assistant-president to the High and Holy Priesthood, in the Church of the Latter-day Saints.\\u201d[3](#f3) On April 3, 1836 Oliver was with Joseph as priesthood key

KeyboardInterrupt: 

In [12]:
with open(output_path + '/question-answers.csv', 'a') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Question', 'Results', 'Answer'])  # header row
    
for key,value in file_data.items():
  with open(output_path + '/question-answers.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
    # for each question, the file should contain a line like this:
    writer.writerow([key, value[0], value[1]])

print('End')

End
